In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import arviz as az
import pymc as pm
import corner
import scipy.stats as st
from statistics import mean, stdev
from collections import Counter

In [ ]:
file_path = 'Datasets/mt_genes_metadata.csv'

met = pd.read_csv(file_path)

pro_code=met[met['gene_type']=='protein_coding']
intpro=pro_code['gene_name'].tolist()
intpro=['MT-ND1','MT-ND2','MT-ND3','MT-ND4','MT-ND4L','MT-ND5','MT-ND6', 'MT-CO1', 'MT-CO2','MT-CO3', 'MT-ATP6', 'MT-ATP8', 'MT-CYB']
intpro

In [ ]:
file_path = 'Datasets/Donor1_CD4_Genes.csv'

p14 = pd.read_csv(file_path)

p14.drop(columns=p14.columns[0], axis=1, inplace=True)

p14_int=pd.DataFrame([p14[i] for i in intpro]).T

file_path = 'Datasets/Donor1_CD8_Genes.csv'

p18 = pd.read_csv(file_path)

p18.drop(columns=p18.columns[0], axis=1, inplace=True)

p18_int=pd.DataFrame([p18[i] for i in intpro]).T

file_path = 'Datasets/Donor2_CD4_Genes.csv'

p24 = pd.read_csv(file_path)

p24.drop(columns=p24.columns[0], axis=1, inplace=True)

p24_int=pd.DataFrame([p24[i] for i in intpro]).T

file_path = 'Datasets/Donor2_CD8_Genes.csv'

p28 = pd.read_csv(file_path)

p28.drop(columns=p28.columns[0], axis=1, inplace=True)

p28_int=pd.DataFrame([p28[i] for i in intpro]).T

total=pd.concat([p14,p18,p24,p28])

total_int=pd.concat([p14_int,p18_int, p24_int,p28_int])

totaln=len(total_int)


In [ ]:
total_sampto=total.sum() #total number of each gene
totalgene=total_sampto.sum() #total number of genes


In [ ]:
#number of DNA strands in each file
p14n=len(p14_int)
p18n=len(p18_int)
p24n=len(p24_int)
p28n=len(p28_int)

In [ ]:
#Calculates the number of genes for each DNA
p14_allsampto=p14.sum(axis=1)
p18_allsampto=p18.sum(axis=1)
p24_allsampto=p24.sum(axis=1)
p28_allsampto=p28.sum(axis=1)

In [ ]:
#Calculates the total number of interested genes for each DNA
p14_intsampto=p14_int.sum(axis=1)
p18_intsampto=p18_int.sum(axis=1)
p24_intsampto=p24_int.sum(axis=1)
p28_intsampto=p28_int.sum(axis=1)

Normalising data by 100/(total interested genes - tested gene) 

In [ ]:
r=100
uem14int_list=[]
for j in intpro:
    uem14int_sample_list=[]
    for i in range(p14n): 
        if p14_intsampto[i]==0:
            pass
        else:
            norm_cons=(p14_intsampto[i]-p14_int[j][i])/r #inidividual normalisation constant
            x=p14_int[j][i]/norm_cons
            uem=(r-1)/(x+r-1)
            uem14int_sample_list.append(uem)
    uem14int_list.append(mean(uem14int_sample_list))
    #the following plots the distribution of the UEM 
    fig, ax = plt.subplots(1, 1) 
    ax.set_title('pat1_4'+''+j)
    ax.hist(uem14int_sample_list,bins=100)
    ax.axvline(mean(uem14int_sample_list), color='pink')
    plt.show()
uem14int_list=dict(zip(intpro,uem14int_list))
df14int=len(uem14int_sample_list)-1

uem18int_list=[]
for j in intpro:
    uem18int_sample_list=[]
    for i in range(p18n): 
        if p18_intsampto[i]==0:
            pass
        else:
            norm_cons=(p18_intsampto[i]-p18_int[j][i])/r #inidividual normalisation constant
            x=p18_int[j][i]/norm_cons
            uem=(r-1)/(x+r-1)
            uem18int_sample_list.append(uem)
    uem18int_list.append(mean(uem18int_sample_list))
    #the following plots the distribution of the UEM 
    fig, ax = plt.subplots(1, 1) 
    ax.set_title('pat1_8'+''+j)
    ax.hist(uem18int_sample_list,bins=100)
    ax.axvline(mean(uem18int_sample_list), color='pink')
    plt.show()
uem18int_list=dict(zip(intpro,uem18int_list))
df18int=len(uem18int_sample_list)-1

uem24int_list=[]
for j in intpro:
    uem24int_sample_list=[]
    for i in range(p24n): 
        if p24_intsampto[i]==0:
            pass
        else:
            norm_cons=(p24_intsampto[i]-p24_int[j][i])/r #inidividual normalisation constant
            x=p24_int[j][i]/norm_cons
            uem=(r-1)/(x+r-1)
            uem24int_sample_list.append(uem)
    uem24int_list.append(mean(uem24int_sample_list))
    #the following plots the distribution of the UEM 
    fig, ax = plt.subplots(1, 1) 
    ax.set_title('pat2_4'+''+j)
    ax.hist(uem24int_sample_list,bins=100)
    ax.axvline(mean(uem24int_sample_list), color='pink')
    plt.show()
uem24int_list=dict(zip(intpro,uem24int_list))
df24int=len(uem24int_sample_list)-1

uem28int_list=[]
for j in intpro:
    uem28int_sample_list=[]
    for i in range(p28n): 
        if p28_intsampto[i]==0:
            pass
        else:
            norm_cons=(p28_intsampto[i]-p28_int[j][i])/r #inidividual normalisation constant
            x=p28_int[j][i]/norm_cons
            uem=(r-1)/(x+r-1)
            uem28int_sample_list.append(uem)
    uem28int_list.append(mean(uem28int_sample_list))
    #the following plots the distribution of the UEM 
    fig, ax = plt.subplots(1, 1) 
    ax.set_title('pat2_8'+''+j)
    ax.hist(uem28int_sample_list,bins=100)
    ax.axvline(mean(uem28int_sample_list), color='pink')
    plt.show()
uem28int_list=dict(zip(intpro,uem28int_list))
df28int=len(uem28int_sample_list)-1

In [35]:

p14_intsampto[0]-p14_int['MT-ND1'][0]

219

χ² test: H0: Follows nbin, H1: Doesn't Follow

In [36]:
#p1_4_int
cv=st.chi2.ppf(0.99,df14int)
chi1_p14int_ts=[]
chi1_p14int_res=[]
for j in intpro:
    ts=0
    p=uem14int_list[j]
    for i in range(p14n):
        if p14_intsampto[i]==0:
            pass
        else:
            r=100
            norm_cons=(p14_intsampto[i]-p14_int[j][i])/r #inidividual normalisation constant
            o=p14_int[j][i]/norm_cons
            e=r*(1-p)/p
            ts+=(e-o)**2/e
    chi1_p14int_ts.append(ts)
    chi1_p14int_res.append(ts-cv)
chi_p14int_res=dict(zip(intpro,chi1_p14int_res))
chi_p14int_res

{'MT-ND1': 188.13309573386096,
 'MT-ND2': 235.4059107760031,
 'MT-ND3': -515.8589575636356,
 'MT-ND4': -177.4493335216589,
 'MT-ND4L': 465.75660927061745,
 'MT-ND5': -89.96436570265541,
 'MT-ND6': -334.6154003048682,
 'MT-CO1': 6579.478205871395,
 'MT-CO2': 3987.817528848831,
 'MT-CO3': 1743.1417288763705,
 'MT-ATP6': -301.62476398443505,
 'MT-ATP8': 2881.5967976359025,
 'MT-CYB': 1360.6901137776713}

In [38]:
#p1_8_int
cv=st.chi2.ppf(0.99,df18int)
chi1_p18int_ts=[]
chi1_p18int_res=[]
for j in intpro:
    ts=0
    p=uem18int_list[j]
    for i in range(p18n):
        if p18_intsampto[i]==0:
            pass
        else:
            r=100
            norm_cons=(p18_intsampto[i]-p18_int[j][i])/r #inidividual normalisation constant
            o=p18_int[j][i]/norm_cons
            e=r*(1-p)/p
            ts+=(e-o)**2/e
    chi1_p18int_ts.append(ts)
    chi1_p18int_res.append(ts-cv)
chi_p18int_res=dict(zip(intpro,chi1_p18int_res))
chi_p18int_res

{'MT-ND1': -400.8239397629927,
 'MT-ND2': -523.0948209476733,
 'MT-ND3': -928.0234415859977,
 'MT-ND4': -737.7250526281971,
 'MT-ND4L': -483.8478235965688,
 'MT-ND5': -498.055638605234,
 'MT-ND6': -577.3353386718879,
 'MT-CO1': 4206.06636339176,
 'MT-CO2': 1097.9205846850905,
 'MT-CO3': 426.8717768070528,
 'MT-ATP6': -844.8878225330782,
 'MT-ATP8': 1306.0738377252915,
 'MT-CYB': 293.72321453059294}

In [45]:
#p2_4_int
cv=st.chi2.ppf(0.99,df24int)
chi1_p24int_ts=[]
chi1_p24int_res=[]
for j in intpro:
    ts=0
    p=uem24int_list[j]
    for i in range(p24n):
        if p24_intsampto[i]==0:
            pass
        else:
            r=100
            norm_cons=(p24_intsampto[i]-p24_int[j][i])/r #inidividual normalisation constant
            o=p24_int[j][i]/norm_cons
            e=r*(1-p)/p
            ts+=(e-o)**2/e
    chi1_p24int_ts.append(ts)
    chi1_p24int_res.append(ts-cv)
chi_p24int_res=dict(zip(intpro,chi1_p24int_res))
chi_p24int_res

{'MT-ND1': 1773.5286007159775,
 'MT-ND2': 1588.816660793328,
 'MT-ND3': 118.90738769486416,
 'MT-ND4': 278.42666224593177,
 'MT-ND4L': 506.2069354833775,
 'MT-ND5': 1308.777439063243,
 'MT-ND6': 3746.270959062792,
 'MT-CO1': 9896.85842052882,
 'MT-CO2': 2644.7522835677537,
 'MT-CO3': 1752.0831990236884,
 'MT-ATP6': 336.08017458125687,
 'MT-ATP8': 2539.3549424613475,
 'MT-CYB': 1982.8789236602288}

In [40]:
#p2_8_int
cv=st.chi2.ppf(0.99,df28int)
chi1_p28int_ts=[]
chi1_p28int_res=[]
for j in intpro:
    ts=0
    p=uem28int_list[j]
    for i in range(p28n):
        if p28_intsampto[i]==0:
            pass
        else:
            r=100
            norm_cons=(p28_intsampto[i]-p28_int[j][i])/r #inidividual normalisation constant
            o=p28_int[j][i]/norm_cons
            e=r*(1-p)/p
            ts+=(e-o)**2/e
    chi1_p28int_ts.append(ts)
    chi1_p28int_res.append(ts-cv)
chi_p28int_res=dict(zip(intpro,chi1_p28int_res))
chi_p28int_res

{'MT-ND1': 336.86994018607515,
 'MT-ND2': -161.10527533567983,
 'MT-ND3': -162.5061471830502,
 'MT-ND4': -172.20962730353222,
 'MT-ND4L': -269.92991451957505,
 'MT-ND5': -1.3499787470334468,
 'MT-ND6': 1327.1850098594466,
 'MT-CO1': 3432.822658214908,
 'MT-CO2': 875.8190210510154,
 'MT-CO3': 240.38474418187138,
 'MT-ATP6': -598.3571687561042,
 'MT-ATP8': 543.2387239914858,
 'MT-CYB': 296.3294203761777}

Normalising data: Normalising data by 1000/(total genes - tested gene) 

and then computing Unbiased estimate of neg binom proportion

Plotting the histograms of the UEM to estimate a prior for the UEM for posterior checks.

In [ ]:
r=1000
uem14_list=[]
for j in intpro:
    uem_sample_list=[]
    for i in range(p14n):   
        norm_cons=(p14_allsampto[i]-p14[j][i])/r #inidividual normalisation constant
        x=p14[j][i]/norm_cons
        uem=(r-1)/(x+r-1)
        uem_sample_list.append(uem)
    uem14_list.append(mean(uem_sample_list))
    #the following plots the distribution of the UEM 
    fig, ax = plt.subplots(1, 1) 
    ax.set_title('pat1_4'+''+j)
    ax.hist(uem_sample_list,bins=100)
    ax.axvline(mean(uem_sample_list), color='pink')
    plt.show()
uem14_list=dict(zip(intpro,uem14_list))

uem18_list=[]
for j in intpro:
    uem_sample_list=[]
    for i in range(p18n):   
        norm_cons=(p18_allsampto[i]-p18[j][i])/r #inidividual normalisation constant
        x=p18[j][i]/norm_cons
        uem=(r-1)/(x+r-1)
        uem_sample_list.append(uem)
    #the following plots the distribution of the UEM 
    fig, ax = plt.subplots(1, 1) 
    ax.set_title('pat1_8'+''+j)
    ax.hist(uem_sample_list,bins=100)
    ax.axvline(mean(uem_sample_list), color='pink')
    plt.show()
    uem18_list.append(mean(uem_sample_list))
uem18_list=dict(zip(intpro,uem18_list))

uem24_list=[]
for j in intpro:
    uem_sample_list=[]
    for i in range(p24n):   
        norm_cons=(p24_allsampto[i]-p24[j][i])/r #inidividual normalisation constant
        x=p24[j][i]/norm_cons
        uem=(r-1)/(x+r-1)
        uem_sample_list.append(uem)
    uem24_list.append(mean(uem_sample_list))
    #the following plots the distribution of the UEM 
    fig, ax = plt.subplots(1, 1) 
    ax.set_title('pat2_4'+''+j)
    ax.hist(uem_sample_list,bins=100)
    ax.axvline(mean(uem_sample_list), color='pink')
    plt.show()
uem24_list=dict(zip(intpro,uem24_list))

uem28_list=[]
for j in intpro:
    uem_sample_list=[]
    for i in range(p28n):   
        norm_cons=(p28_allsampto[i]-p28[j][i])/r #inidividual normalisation constant
        x=p28[j][i]/norm_cons
        uem=(r-1)/(x+r-1)
        uem_sample_list.append(uem)
    uem28_list.append(mean(uem_sample_list))
    #the following plots the distribution of the UEM 
    fig, ax = plt.subplots(1, 1) 
    ax.set_title('pat2_8'+''+j)
    ax.hist(uem_sample_list,bins=100)
    ax.axvline(mean(uem_sample_list), color='pink')
    plt.show()
uem28_list=dict(zip(intpro,uem28_list))
uem28_list

χ² test: H0: Follows nbin, H1: Doesn't Follow

In [ ]:
#p1_4
df14=p14n-1
cv=st.chi2.ppf(0.99,df14)
chi1_p14_ts=[]
chi1_p14_res=[]
for j in intpro:
    ts=0
    p=uem14_list[j]
    for i in range(p14n):
        r=1000
        norm_cons=(p14_allsampto[i]-p14[j][i])/r #inidividual normalisation constant
        o=p14[j][i]/norm_cons
        e=r*(1-p)/p
        ts+=(e-o)**2/e
    chi1_p14_ts.append(ts)
    chi1_p14_res.append(ts-cv)
chi_p14_res=dict(zip(intpro,chi1_p14_res))
chi_p14_res

In [ ]:
#p1_8
df18=p18n-1
cv=st.chi2.ppf(0.99,df18)
chi1_p18_ts=[]
chi1_p18_res=[]
for j in intpro:
    ts=0
    p=uem18_list[j]
    for i in range(p18n):
        r=1000
        norm_cons=(p18_allsampto[i]-p18[j][i])/r #inidividual normalisation constant
        o=p18[j][i]/norm_cons
        e=r*(1-p)/p
        ts+=(e-o)**2/e
    chi1_p18_ts.append(ts)
    chi1_p18_res.append(ts-cv)
chi_p18_res=dict(zip(intpro,chi1_p18_res))
chi_p18_res

In [ ]:
#p2_4
df24=p24n-1
cv=st.chi2.ppf(0.99,df24)
chi1_p24_ts=[]
chi1_p24_res=[]
for j in intpro:
    ts=0
    p=uem24_list[j]
    for i in range(p24n):
        r=1000
        norm_cons=(p24_allsampto[i]-p24[j][i])/r #inidividual normalisation constant
        o=p24[j][i]/norm_cons
        e=r*(1-p)/p
        ts+=(e-o)**2/e
    chi1_p24_ts.append(ts)
    chi1_p24_res.append(ts-cv)
chi_p24_res=dict(zip(intpro,chi1_p24_res))
chi_p24_res

In [ ]:
#p2_8
df28=p28n-1
cv=st.chi2.ppf(0.99,df28)
chi1_p28_ts=[]
chi1_p28_res=[]
for j in intpro:
    ts=0
    p=uem28_list[j]
    for i in range(p28n):
        r=1000
        norm_cons=(p28_allsampto[i]-p28[j][i])/r #inidividual normalisation constant
        o=p28[j][i]/norm_cons
        e=r*(1-p)/p
        ts+=(e-o)**2/e
    chi1_p28_ts.append(ts)
    chi1_p28_res.append(ts-cv)
chi_p28_res=dict(zip(intpro,chi1_p28_res))
chi_p28_res

In [ ]:
chi_p14_resdf=pd.Series(chi_p14_res)
chi_p14_resdf=pd.DataFrame(chi_p14_resdf,columns=['P1_4'])
chi_p18_resdf=pd.Series(chi_p18_res)
chi_p18_resdf=pd.DataFrame(chi_p18_resdf,columns=['P1_8'])
chi_p24_resdf=pd.Series(chi_p24_res)
chi_p24_resdf=pd.DataFrame(chi_p24_resdf,columns=['P2_4'])
chi_p28_resdf=pd.Series(chi_p28_res)
chi_p28_resdf=pd.DataFrame(chi_p28_resdf,columns=['P2_8'])
overall_chi_res=pd.concat([chi_p14_resdf,chi_p18_resdf,chi_p24_resdf, chi_p28_resdf], axis=1)

In [ ]:
overall_chi_res

The following started with trying a Partial correlation analysis by plotting each gene against 'non-MT' UMI. but the rvalues were rather low.

Instead, I experimented a bit and plotted the y against √xy where y=interested gene, x='non-MT'

In [ ]:
x=p14['non-MT']
r14_l=[]
for i in intpro:
    fig, ax = plt.subplots(1, 1) 
    y=p14[i]
    ax.set_title('p14'+''+i)
    lin_x=np.sqrt(x*y)
    lin_y=y
    ax.scatter(lin_x,lin_y, s=1)
    M=max(lin_x)
    m=min(lin_x)
    lin=st.linregress(lin_x,lin_y)
    l=np.array([m,M])
    ax.plot(l,lin.intercept+lin.slope*l,color='red')
    r14_l.append(lin.rvalue)
    plt.show()
r14_l


In [ ]:
x=p18['non-MT']
r18_l=[]
for i in intpro:
    fig, ax = plt.subplots(1, 1) 
    y=p18[i]
    ax.set_title('p18'+''+i)
    lin_x=np.sqrt(x*y)
    lin_y=y
    ax.scatter(lin_x,lin_y, s=1)
    M=max(lin_x)
    m=min(lin_x)
    lin=st.linregress(lin_x,lin_y)
    l=np.array([m,M])
    ax.plot(l,lin.intercept+lin.slope*l,color='red')
    r18_l.append(lin.rvalue)
    plt.show()
r18_l


In [ ]:
x=p24['non-MT']
r24_l=[]
for i in intpro:
    fig, ax = plt.subplots(1, 1) 
    y=p24[i]
    ax.set_title('p24'+''+i)
    lin_x=np.sqrt(x*y)
    lin_y=y
    ax.scatter(lin_x,lin_y, s=1)
    M=max(lin_x)
    m=min(lin_x)
    lin=st.linregress(lin_x,lin_y)
    l=np.array([m,M])
    ax.plot(l,lin.intercept+lin.slope*l,color='red')
    r24_l.append(lin.rvalue)
    plt.show()
r24_l


In [ ]:
x=p28['non-MT']
r28_l=[] #rvalues
for i in intpro:
    fig, ax = plt.subplots(1, 1) 
    y=p28[i]
    ax.set_title('p28'+''+i)
    lin_x=np.sqrt(x*y)
    lin_y=y
    ax.scatter(lin_x,lin_y, s=1)
    M=max(lin_x)
    m=min(lin_x)
    lin=st.linregress(lin_x,lin_y)
    l=np.array([m,M])
    ax.plot(l,lin.intercept+lin.slope*l,color='red')
    r28_l.append(lin.rvalue)
    plt.show()
r28_l


In [ ]:
x=p14['non-MT']
resi14=pd.DataFrame()
i='MT-ND1'
fig, ax = plt.subplots(1, 1) 
y=p14[i]
lin_x=x
lin_y=y
lin=st.linregress(lin_x,lin_y)
fit=lin.intercept+lin.slope*lin_x
resdf=pd.DataFrame(lin_y-fit)
resdf.columns=[i]
resi14=resi14.join(resdf,how='outer')
ax.set_title('p14'+i)
ax.scatter(lin_x,lin_y)
ax.plot(lin_x,fit, color='red',label=lin.rvalue)
plt.legend()
plt.show()
resi14

Calculating the residuals as part of the analysis (and also plotting to check residuals)

In [ ]:
x=p14['non-MT']
resi14=pd.DataFrame()
for i in intpro:
    fig, ax = plt.subplots(1, 1) 
    y=p14[i]
    lin_x=x
    lin_y=y
    lin=st.linregress(lin_x,lin_y)
    fit=lin.intercept+lin.slope*lin_x
    resdf=pd.DataFrame(lin_y-fit)
    resdf.columns=[i]
    resi14=resi14.join(resdf,how='outer')
    ax.scatter(lin_x,lin_y)
    ax.plot(lin_x,fit, color='red',label=lin.rvalue)
    ax.set_title('p14'+i)
    plt.legend()
    plt.show()
resi14

Following are the plots of the (untransformed) residuals against another, with the linear regression plotted and corresponding r-value 

In [ ]:
tet=[]
j='MT-ND1'
res_x=resi14[j]
res_col=[]
for i in intpro:
    fig, ax = plt.subplots(1, 1) 
    res_y=resi14[i]
    ax.set_title('p14'+i+'vs'+j)
    lin_x=res_x
    lin_y=res_y
    ax.scatter(lin_x,lin_y, s=1)
    M=max(lin_x)
    m=min(lin_x)
    lin=st.linregress(lin_x,lin_y)
    l=np.array([m,M])
    ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
    plt.legend()
    plt.show()

In [ ]:
res_x=resi14['MT-ND1']
res_y=resi14['MT-ND2']
lin_x=np.sqrt(np.abs((res_x*res_y)**2/(0.01)**2-1))
lin_y=np.abs(res_y)


Checkpoint: 060624

To do next: Normalise the residuals somehow (the one where you log, refer to checkpoint

In [ ]:
#plotting |y| against ln(|xy|/k+1)
j='MT-ND1'
res_x=resi14[j]
i='MT-ND2'
fig, ax = plt.subplots(1, 1) 
res_y=resi14[i]
ax.set_title('p14'+i+'against'+i+'*'+j)
lin_x=np.abs(res_x*res_y)
lin_y=np.abs(res_y)
ax.scatter(lin_x,lin_y, s=1)
M=max(lin_x)
m=min(lin_x)
lin=st.linregress(lin_x,lin_y)
l=np.array([m,M])
ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
hyp_x=np.linspace(m,M,1000)
hyp_y=np.log(hyp_x/(res14_D2_sd**1.5)+1)
ax.plot(hyp_x,hyp_y*res14_D2_sd,color='pink')
plt.legend()
plt.show()

In [ ]:
#linreg |y| against ln(|xy|/k+1)
j='MT-ND1'
res_x=resi14[j]
i='MT-ND2'
fig, ax = plt.subplots(1, 1) 
res_y=resi14[i]
ax.set_title('p14'+i+'against'+i+'*'+j)
lin_x1=np.abs(res_x*res_y)
lin_y=np.abs(res_y)
lin_x=np.log(lin_x1/(res14_D2_sd**2)+2)
ax.scatter(lin_x,lin_y, s=1)
M=max(lin_x)
m=min(lin_x)
lin=st.linregress(lin_x,lin_y)
l=np.array([m,M])
ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
plt.legend()
plt.show()

In [ ]:
#linreg |y| against -1/x^n
j='MT-ND1'
res_x=resi14[j]
i='MT-ND2'
fig, ax = plt.subplots(1, 1) 
res_y=resi14[i]
ax.set_title('p14'+i+'against'+i+'*'+j)
lin_x1=np.abs(res_x*res_y)
lin_y=np.abs(res_y)
lin_x=1/(x+1)**2
ax.scatter(lin_x,lin_y, s=1)
M=max(lin_x)
m=min(lin_x)
lin=st.linregress(lin_x,lin_y)
l=np.array([m,M])
ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
plt.legend()
plt.show()

In [ ]:
#test plotting |y| against -1/|xy|
j='MT-ND1'
res_x=resi14[j]
i='MT-ND2'
fig, ax = plt.subplots(1, 1) 
res_y=resi14[i]
ax.set_title('p14'+i+'against'+i+'*'+j)
lin_x=np.abs(res_x*res_y)
lin_y=np.abs(res_y)
ax.scatter(lin_x,lin_y, s=1)
M=max(lin_x)
m=min(lin_x)
#lin=st.linregress(lin_x,lin_y)
l=np.array([m,M])
#ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
hyp_x=np.linspace(m,M,1000)
hyp_y=-500/((hyp_x+10*res14_D2_sd)**1.1)+12
ax.plot(hyp_x,hyp_y,color='pink')
#plt.legend()
plt.show()

In [ ]:
#test plotting |y| against -1/|xy|
j='MT-ND1'
res_x=resi14[j]
i='MT-ND6'
fig, ax = plt.subplots(1, 1) 
res_y=resi14[i]
ax.set_title('p14'+i+'against'+i+'*'+j)
lin_x=np.abs(res_x*res_y)
lin_y=np.abs(res_y)
ax.scatter(lin_x,lin_y, s=1)
M=max(lin_x)
m=min(lin_x)
#lin=st.linregress(lin_x,lin_y)
l=np.array([m,M])
#ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
hyp_x=np.linspace(m,M,1000)
hyp_y=-100/((hyp_x+12)**1.1)+6
ax.plot(hyp_x,hyp_y,color='pink')
#plt.legend()
plt.show()

With hyp_y=-500/((hyp_x+30)**1.1)+12 ≡ -β/((hyp_x+a)^n)+γ, all that's left is to fine a good value/formula for a & n for the linreg

After some testing, we can put a=10*s.d
n=1.1 for now

In [ ]:
#lin reg |y| against -1/(|xy|+k)
j='MT-ND1'
res_x=resi14[j]
for i in intpro:
    sd=stdev(resi14[i])
    fig, ax = plt.subplots(1, 1) 
    res_y=resi14[i]
    ax.set_title('p14'+i+'against'+i+'*'+j)
    lin_x1=np.abs(res_x*res_y)
    lin_y=np.abs(res_y)
    lin_x=-1/((lin_x1+10*sd)**1.2)
    ax.scatter(lin_x,lin_y, s=1)
    M=max(lin_x)
    m=min(lin_x)
    lin=st.linregress(lin_x,lin_y)
    l=np.array([m,M])
    ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
    plt.legend()
    plt.show()

In [ ]:
#test plotting |y| against |xy|
j='MT-ND1'
res_x=resi14[j]
for i in intpro:
    fig, ax = plt.subplots(1, 1) 
    res_y=resi14[i]
    ax.set_title('p14'+i+'against'+i+'*'+j)
    lin_x=np.abs(res_x*res_y)
    lin_y=np.abs(res_y)
    ax.scatter(lin_x,lin_y, s=1)
    M=max(lin_x)
    m=min(lin_x)
    lin=st.linregress(lin_x,lin_y)
    l=np.array([m,M])
    ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
    hyp_x=np.linspace(m,M,1000)
    hyp_y=np.log(hyp_x+1)
    ax.plot(hyp_x,hyp_y,color='pink')
    plt.legend()
    plt.show()

In [ ]:
#lin reg |y| against √|xy^2/k-1|
j='MT-ND1'
res_x=resi14[j]
for i in intpro:
    fig, ax = plt.subplots(1, 1) 
    res_y=resi14[i]
    ax.set_title('p14'+i+'against'+i+'*'+j)
    lin_x=
    lin_y=np.abs(res_y)
    ax.scatter(lin_x,lin_y, s=1)
    M=max(lin_x)
    m=min(lin_x)
    lin=st.linregress(lin_x,lin_y)
    l=np.array([m,M])
    ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
    plt.legend()
    plt.show()


In [ ]:
#plotting |y| against √|xy|
j='MT-ND1'
res_x=resi14[j]
for i in intpro:
    fig, ax = plt.subplots(1, 1) 
    res_y=resi14[i]
    ax.set_title('p14'+i+'against'+i+'*'+j)
    lin_x=np.sqrt(np.abs(res_x*res_y))
    lin_y=np.abs(res_y)
    ax.scatter(lin_x,lin_y, s=1)
    M=max(lin_x)
    m=min(lin_x)
    lin=st.linregress(lin_x,lin_y)
    l=np.array([m,M])
    ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
    plt.legend()
    plt.show()


In [ ]:
#plotting |y| against sigmoid(|xy|)
j='MT-ND1'
res_x=resi14[j]
for i in intpro:
    fig, ax = plt.subplots(1, 1) 
    res_y=resi14[i]
    ax.set_title('p14'+i+'against'+i+'*'+j)
    trans_x=np.abs(res_x*res_y)
    lin_x=
    lin_y=np.abs(res_y)
    ax.scatter(lin_x,lin_y, s=1)
    M=max(lin_x)
    m=min(lin_x)
    lin=st.linregress(lin_x,lin_y)
    l=np.array([m,M])
    ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
    plt.legend()
    plt.show()

Residuals plot: y against x*y

In [ ]:
r_resi14_l=[] #r-values for transformed residuals
for j in intpro:
    res_x=resi14[j]
    res_col=[]
    for i in intpro:
        fig, ax = plt.subplots(1, 1) 
        res_y=resi14[i]
        ax.set_title('p14'+i+'against'+i+'*'+j)
        lin_x=res_x*res_y
        lin_y=res_y
        ax.scatter(lin_x,lin_y, s=1)
        M=max(lin_x)
        m=min(lin_x)
        lin=st.linregress(lin_x,lin_y)
        l=np.array([m,M])
        ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
        res_col.append(lin.rvalue)
        plt.legend()
        plt.show()
    r_resi14_l.append(res_col)  

Plotting residuals: y against |xy|

In [ ]:
r_resi14_l=[] #r-values for transformed residuals
for j in intpro:
    res_x=resi14[j]
    res_col=[]
    for i in intpro:
        fig, ax = plt.subplots(1, 1) 
        res_y=resi14[i]
        ax.set_title('p14'+i+'against'+i+'*'+j)
        lin_x=res_x
        lin_y=np.abs(res_y*res_x)
        ax.scatter(lin_x,lin_y, s=1)
        M=max(lin_x)
        m=min(lin_x)
        lin=st.linregress(lin_x,lin_y)
        l=np.array([m,M])
        ax.plot(l,lin.intercept+lin.slope*l,color='red',label=lin.rvalue)
        res_col.append(lin.rvalue)
        plt.legend()
        plt.show()
    r_resi14_l.append(res_col)  

In [ ]:
resi18=pd.DataFrame()
x=p18['non-MT']
for i in intpro:
    y=p18[i]
    lin_x=np.sqrt(x*y)
    lin_y=y
    lin=st.linregress(lin_x,lin_y)
    fit=lin.intercept+lin.slope*lin_x
    resi18=resi18.join(pd.DataFrame(lin_y-fit), how='outer')
resi18

In [ ]:
resi14['MT-ND1']

In [ ]:
#function to calc and include r-value onto the plots
def rval(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, _ = st.pearsonr(x, y)
    ax = ax or plt.gca()
    ax.annotate(f'ρ = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)

In [ ]:
res_plot=sns.pairplot(resi14,kind='reg', diag_kind='kde',plot_kws={'line_kws':{'color':'red'}},corner=True)
res_plot.map_lower(rval)
plt.show()

rvalue looks very bleak...
Transforming the residuals to a similar way to above

In [ ]:
r_resi14_l=[] #r-values for transformed residuals
for j in intpro:
    res_x=resi14[j]
    res_col=[]
    for i in intpro:
        fig, ax = plt.subplots(1, 1) 
        y=resi14[i]
        ax.set_title('p14'+j+'against'+'sqrt'+'('+i+j+')')
        lin_x=x*y
        lin_y=y
        ax.scatter(lin_x,lin_y, s=1)
        M=max(lin_x)
        m=min(lin_x)
        lin=st.linregress(lin_x,lin_y)
        l=np.array([m,M])
        ax.plot(l,lin.intercept+lin.slope*l,color='red')
        res_col.append(lin.rvalue)
        plt.show()
    r_resi14_l.append(res_col)  

In [ ]:
tex=resi14['MT-ND1']*resi14['MT-ND2']
tey=resi14['MT-ND1']
plt.scatter(tex,tey)
telin=st.linregress(tex,tey)
M=max(tex)
m=min(tex)
tel=np.array([m,M])
plt.plot(tel,telin.intercept+telin.slope*tel,color='red')
plt.show()
telin.rvalue

Log?

In [ ]:
tex=np.log(resi14['MT-ND1']*resi14['MT-ND2']+4)
tey=resi14['MT-ND1']
plt.scatter(tex,tey)
telin=st.linregress(tex,tey)
M=max(tex)
m=min(tex)
tel=np.array([m,M])
plt.plot(tel,telin.intercept+telin.slope*tel,color='red')
plt.show()
telin.rvalue

Hyperbola idea but instead, consider reciprocal function and take negative, then translate upwards. 

i.e: -β_1 (xy-k)**(-n) + β_0

In [ ]:
tex=-1/(resi14['MT-ND1']*resi14['MT-ND2']+0.1)
tey=resi14['MT-ND1']
plt.scatter(tex,tey)
telin=st.linregress(tex,tey)
M=max(tex)
m=min(tex)
tel=np.array([m,M])
plt.plot(tel,telin.intercept+telin.slope*tel,color='red')
plt.show()
telin.rvalue

Below are a few code trying to fit a hyperbola (incomplete)

In [ ]:
np.sqrt((resi14['MT-ND1']*resi14['MT-ND2']+1)**2/0.05-1)

In [ ]:
tex=np.sqrt((resi14['MT-ND1']*resi14['MT-ND2'])**2/0.05-1)
np.isnan(tex).any()

In [ ]:
tex=np.sqrt((resi14['MT-ND1']*resi14['MT-ND2']+2)**2-0.25)
tey=resi14['MT-ND1']
plt.scatter(tex,tey)
telin=st.linregress(tex,tey)
M=max(tex)
m=min(tex)
tel=np.array([m,M])
plt.plot(tel,telin.intercept+telin.slope*tel,color='red')
plt.show()
telin.rvalue

In [ ]:
tex=np.sqrt(resi14['MT-ND1']*resi14['MT-ND2'])
tey=resi14['MT-ND1']
plt.scatter(tex,tey)
telin=st.linregress(tex,tey)
M=max(tex)
m=min(tex)
tel=np.array([m,M])
plt.plot(tel,telin.intercept+telin.slope*tel,color='red')
plt.show()

In [ ]:
m_tex=abs(min(resi14['MT-ND1']*resi14['MT-ND2']))
tex=np.sqrt(resi14['MT-ND1']*resi14['MT-ND2']+m_tex)
tey=resi14['MT-ND1']
plt.scatter(tex,tey)
telin=st.linregress(tex,tey)
M=max(tex)
m=min(tex)
tel=np.array([m,M])
plt.plot(tel,telin.intercept+telin.slope*tel,color='red')
plt.show()
telin.rvalue

Code to plot the transformed residuals (incomplete)

In [ ]:
r_resi14_l=[] #r-values for transformed residuals
for j in intpro:
    res_x=resi14[j]
    res_col=[]
    for i in intpro:
        fig, ax = plt.subplots(1, 1) 
        res_y=resi14[i]
        ax.set_title('p14'+j+'against'+'sqrt'+'('+i+j+')')
        lin_x=np.sqrt(res_x*res_y)
        lin_y=res_y
        ax.scatter(lin_x,lin_y, s=1)
        M=max(lin_x)
        m=min(lin_x)
        lin=st.linregress(lin_x,lin_y)
        l=np.array([m,M])
        ax.plot(l,lin.intercept+lin.slope*l,color='red')
        res_col.append(lin.rvalue)
        plt.show()
    r_resi14_l.append(res_col)  

In [ ]:
r_resi14_l

In [ ]:
resi24=pd.DataFrame()
for i in intpro:
    y=p24[i]
    lin_x=np.sqrt(x*y)
    lin_y=y
    lin=st.linregress(lin_x,lin_y)
    fit=lin.intercept+lin.slope*lin_x
    resi24=resi24.join(pd.DataFrame(lin_y-fit), how='outer')
resi24

In [ ]:
resi28=pd.DataFrame()
for i in intpro:
    y=p28[i]
    lin_x=np.sqrt(x*y)
    lin_y=y
    lin=st.linregress(lin_x,lin_y)
    fit=lin.intercept+lin.slope*lin_x
    resi28=resi28.join(pd.DataFrame(lin_y-fit), how='outer')
resi28

In [ ]:
x=p28['non-MT']
p28_resi=[]
for i in intpro:
    y=p28[i]
    lin_x=np.sqrt(x*y)
    lin_y=y
    lin=st.linregress(lin_x,lin_y)
    fit=lin.intercept+lin.slope*lin_x
    resi=lin_y-fit
p28_resi

For justification: The following was the original plan: plotting gene against non-MT UMI.

Terrible r-value

In [ ]:
r_l=[]
for i in intpro:
    fig, ax = plt.subplots(1, 1) 
    ax.set_title('pat1_4'+''+i)
    ax.scatter(p14['non-MT'],p14[i],s=1)
    beta=st.linregress(p14['non-MT'], p14[i])
    ax.plot(np.linspace(0,20000,num=5),beta.slope*np.linspace(0,20000,num=5)+beta.intercept,color='red')
    r_l.append(beta.rvalue)
    plt.show()


In [ ]:
r_l